# Simple Impementation of E(n) Equivariant Graph Neural Networks

Original paper https://arxiv.org/pdf/2102.09844.pdf by Victor Garcia Satorras, Emiel Hoogeboom, Max Welling

In [1]:
import time
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

# Load QM9 Dataset

In [2]:
!git clone https://github.com/senya-ashukha/simple-equivariant-gnn.git
%cd simple-equivariant-gnn

fatal: destination path 'simple-equivariant-gnn' already exists and is not an empty directory.
/home/leonid/Загрузки/Telegram Desktop/simple-equivariant-gnn


In [3]:
# QM9 is a dataset for Molecular Property Predictions http://quantum-machine.org/datasets/
# We will predict Highest occupied molecular orbital energy 
# https://en.wikipedia.org/wiki/HOMO_and_LUMO
# We use data loaders from the official repo

from qm9.data_utils import get_data, BatchGraph
train_loader, val_loader, test_loader, charge_scale = get_data(num_workers=1)

# Graph Representation

In [4]:
batch = BatchGraph(iter(train_loader).next(), False, charge_scale)
batch

In the batch: num_graphs 96 num_nodes 1749
> .h 		 a tensor of nodes representations 		shape 1749 x 15
> .x 		 a tensor of nodes positions  			shape 1749 x 3
> .edges 	 a tensor of edges, a fully connected graph 	shape 31134 x 2
> .batch  	 a tensor of graph_ids for each node 		tensor([ 0,  0,  0,  ..., 95, 95, 95])

# Define Equivariant Graph Convs  & GNN

In [5]:
def index_sum(agg_size, source, idx, cuda):
    tmp = torch.zeros((agg_size, source.shape[1]))
    tmp = tmp.cuda() if cuda else tmp
    res = torch.index_add(tmp, 0, idx, source)
    return res

In [6]:
class ConvEGNN(nn.Module):
    def __init__(self, in_dim, hid_dim, cuda=True):
        super().__init__()
        self.hid_dim=hid_dim
        self.cuda = cuda
        
        self.f_e = nn.Sequential(
            nn.Linear(2 * in_dim + 1, hid_dim),
            nn.SiLU(),
            nn.Linear(hid_dim, hid_dim),
            nn.SiLU()
            )

        self.f_inf = nn.Sequential(
            nn.Linear(hid_dim, 1),
            nn.Sigmoid()
            )
        
        self.f_h = nn.Sequential(
            nn.Linear(in_dim + hid_dim, hid_dim),
            nn.SiLU(),
            nn.Linear(hid_dim, hid_dim)
            )

    
    def forward(self, b):
        e_st, e_end = b.edges[:,0], b.edges[:,1]
        dists = torch.norm(b.x[e_st] - b.x[e_end], dim=1).reshape((-1, 1))

        tmp = torch.hstack([b.h[e_st], b.h[e_end], dists])
        m_ij = self.f_e(tmp).reshape((-1, self.hid_dim))
        
        e_ij = self.f_inf(m_ij)
        
        m_i = index_sum(b.h.shape[0], e_ij*m_ij, b.edges[:,0], self.cuda)
        
        b.h = torch.reshape(self.f_h(torch.hstack([b.h, m_i])) + b.h, (-1, self.hid_dim))

        return b

In [7]:
class NetEGNN(nn.Module):
    def __init__(self, in_dim=15, hid_dim=128, out_dim=1, n_layers=7, cuda=True):
        super().__init__()
        self.hid_dim=hid_dim
        
        self.emb = nn.Linear(in_dim, hid_dim) 

        self.gnn = [ConvEGNN(hid_dim, hid_dim, cuda=cuda) for _ in range(n_layers)]
        self.gnn = nn.Sequential(*self.gnn)

        
        self.pre_mlp = nn.Sequential(
            nn.Linear(hid_dim, hid_dim), nn.SiLU(),
            nn.Linear(hid_dim, hid_dim))
        
        self.post_mlp = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(hid_dim, hid_dim), nn.SiLU(),
            nn.Linear(hid_dim, out_dim))

        if cuda: self.cuda()
        self.cuda = cuda
    
    def forward(self, b):
        b.h = self.emb(b.h)
        
        b = self.gnn(b)
        h_nodes = self.pre_mlp(b.h)
        
        h_graph = index_sum(b.nG, h_nodes, b.batch, self.cuda) 
        
        out = self.post_mlp(h_graph)
        return out

In [8]:
epochs = 200
cuda = True

model = NetEGNN(n_layers=7, cuda=cuda)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-16)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, verbose=False)

# Training

In [9]:
print('> start training')

tr_ys = train_loader.dataset.data['homo'] 
me, mad = torch.mean(tr_ys), torch.mean(torch.abs(tr_ys - torch.mean(tr_ys)))

if cuda:
    me = me.cuda()
    mad = mad.cuda()

train_loss = []
val_loss = []
test_loss = []

for epoch in range(epochs):
    print('> epoch %s:' % str(epoch).zfill(3), end=' ', flush=True) 
    start = time.time()

    batch_train_loss = []
    batch_val_loss = []
    batch_test_loss = []

    model.train()
    for batch in train_loader:
        batch = BatchGraph(batch, cuda, charge_scale)
        
        out = model(batch).reshape(-1)
        loss = F.l1_loss(out, (batch.y-me)/mad)

        loss.backward()

        optimizer.step()
        optimizer.zero_grad()
        
        with torch.no_grad():
            loss =  F.l1_loss(out*mad+me, batch.y)

        batch_train_loss += [float(loss.data.cpu().numpy())]  
        
    train_loss += [np.mean(batch_train_loss)/0.001]
    
    print('train %.3f' % train_loss[-1], end=' ', flush=True)
    
    with torch.no_grad():
        model.eval()
        for batch in val_loader:
            batch = BatchGraph(batch, cuda, charge_scale)
            out = model(batch).reshape(-1)
            loss = F.l1_loss(out*mad+me, batch.y).data.cpu().numpy()
            batch_val_loss += [np.mean(loss)]
            
        val_loss += [np.mean(batch_val_loss)/0.001]
        
        print('val %.3f' % val_loss[-1], end=' ', flush=True)
        
        for batch in test_loader:
            batch = BatchGraph(batch, cuda, charge_scale)
            out = model(batch).reshape(-1)
            loss = F.l1_loss(out*mad+me, batch.y).data.cpu().numpy()
            batch_test_loss += [np.mean(loss)]

        test_loss += [np.mean(batch_test_loss)/0.001]
        
    end = time.time()

    print('test %.3f (%.1f sec)' % (test_loss[-1], end-start), flush=True)
    lr_scheduler.step()

> start training
> epoch 000: train 391.683 val 322.151 test 321.186 (130.6 sec)
> epoch 001: train 281.738 val 244.154 test 241.878 (129.7 sec)
> epoch 002: train 223.642 val 205.746 test 204.311 (129.7 sec)
> epoch 003: train 198.256 val 195.704 test 193.652 (128.0 sec)
> epoch 004: train 180.583 val 185.659 test 185.232 (128.1 sec)
> epoch 005: train 166.743 val 173.664 test 171.944 (128.0 sec)
> epoch 006: train 154.025 val 138.538 test 138.583 (127.8 sec)
> epoch 007: train 142.391 val 133.925 test 131.565 (128.3 sec)
> epoch 008: train 133.180 val 122.881 test 123.177 (127.9 sec)
> epoch 009: train 126.694 val 116.769 test 118.137 (129.0 sec)
> epoch 010: train 122.175 val 116.368 test 116.089 (131.9 sec)
> epoch 011: train 115.751 val 113.866 test 113.141 (131.0 sec)
> epoch 012: train 111.131 val 111.591 test 112.609 (131.5 sec)
> epoch 013: train 108.285 val 98.241 test 98.364 (129.0 sec)
> epoch 014: train 103.581 val 115.931 test 114.751 (129.1 sec)
> epoch 015: train 100.25